# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

In [31]:
!pip install nltk

In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import nltk

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [4]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

In [2]:
dataframe = pd.read_csv('data/recipes_sample.csv')
dataframe

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,NaN,this is a delicious soup that i originally fou...,NaN


1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны.

In [3]:
rand_sample_ing = dataframe.sample(n=5)
max_id, max_minutes = 0, 0

for index in rand_sample_ing.index:
    max_id = max(max_id, dataframe['id'][index])
    max_minutes = max(max_minutes, dataframe['minutes'][index])

max_len_id = len([digit for digit in str(max_id)]) + 8
max_len_minutes = len([digit for digit in str(max_minutes)]) + 8

print(f"|{'id':^{max_len_id}}|{'minutes':^{max_len_minutes}}|")
print("|" + "–" * (max_len_id + max_len_minutes + 1) + "|")
for index in rand_sample_ing.index:
    print(f"|{rand_sample_ing['id'][index]:^{max_len_id}}|{rand_sample_ing['minutes'][index]:^{max_len_minutes}}|")

|      id      |  minutes   |
|–––––––––––––––––––––––––––|
|    168184    |     7      |
|    507927    |     40     |
|    22105     |     55     |
|    197018    |     72     |
|    144742    |    1447    |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [4]:
dataframe[dataframe['id'] == 170895]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
15754,leeks and parsnips sauteed or creamed,170895,27,8377,2006-05-31,21.0,this is good sauteed only or creamed. very eas...,9.0


In [5]:
with open('data/steps_sample.xml', 'r') as fp:
    steps_sample = BeautifulSoup(fp, 'xml')

def show_info(recipe_id=-1, name="", steps=[], minutes=0, author_id=0, is_step=False):
    final_str, steps_str = '', steps
    dataframe = pd.read_csv('data/recipes_sample.csv')
    dataset = dataframe[dataframe['id'] == recipe_id]

    if recipe_id != -1:
        name = dataset['name'].item()
        with open('data/steps_sample.xml', 'r') as fp:
            steps_sample = BeautifulSoup(fp, 'xml')
            for recipe in steps_sample.find_all('recipe'):
                if recipe.find("id").text == str(recipe_id):
                    steps_str = [step.text for step in recipe.find_all('step')]
                    break
        minutes = dataset["minutes"].item()
        author_id = dataset['contributor_id'].item()

    final_str += '"' + name.title() + '"\n\n'

    for index in range(len(steps_str)):
        final_str += str(index + 1) + '. ' + steps_str[index].capitalize() + '\n'

    final_str += 10 * "-" + "\nАвтор: " + str(author_id) + '\nСреднее время приготовления: ' + str(minutes) + " минут"

    if is_step:
        return steps_str
    else:
        return final_str

print(show_info(170895))

"Leeks And Parsnips  Sauteed Or Creamed"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an addi

In [6]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут'
)

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [7]:
dataframe[dataframe['id'] == 25082]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
2415,basic whole wheat bread,25082,290,25483,2002-04-13,18.0,"a simple, easy whole wheat bread recipe posted...",NaN


In [8]:
dataset = show_info(25082, is_step=True)
dataset

['proof yeast in half a cup of the water , with about a teaspoon of flour- let it sit , covered , until it becomes bubbly& active',
 'combine the rest of the water with the yeast / water in a large bowl , and gradually add about half of the flour',
 'stir the mixture about 100 times in the same direction , until it is well-mixed and strands are beginning to form',
 'stir in the salt and oil',
 'add the remaining flour half a cup at a time , stirring well , until it is too stiff to stir with a spoon',
 'turn out onto a lightly floured board and knead for about 20 minutes , adding flour as nescessary to keep the dough from sticking to the board',
 'the dough should be springy and nice to work with',
 'when it has been sufficiently kneaded , cover it with a damp cloth for about 10 minutes and wash and grease the bowl lightly',
 'after it has rested , knead the dough a few more times and place it in the bowl , covered with the damp cloth , a plastic bag , and maybe a plate',
 'let the doug

In [9]:
pattern = re.compile(r"([\d]+ (hour[s]*|minute[s]*))")
pattern.findall('\n'.join(dataset))

[('20 minutes', 'minutes'),
 ('10 minutes', 'minutes'),
 ('2 hours', 'hours'),
 ('10 minutes', 'minutes'),
 ('20 minutes', 'minutes'),
 ('30 minutes', 'minutes')]

4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [10]:
pattern = re.compile(r"^this.*[,] ?but")
dess = pd.Series(dataframe.description).dropna()
patt_dess = dess.str.match(pattern)
patt_dess.sum()

865

In [11]:
dess[patt_dess].sample(n=3)

10144    this pie is so easy to make, if you love peanu...
649      this recipe was originally made into a 9" pie,...
10231    this is a recipe that i copied down from a mag...
Name: description, dtype: object

5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [12]:
dataset = show_info(72367, is_step=True)
dataset

['mix butter , flour , 1 / 3 c',
 'sugar and 1-1 / 4 t',
 'vanilla',
 'press into greased 9" springform pan',
 'mix cream cheese , 1 / 4 c',
 'sugar , eggs and 1 / 2 t',
 'vanilla beating until fluffy',
 'pour over dough',
 'combine apples , 1 / 3 c',
 'sugar and cinnamon',
 'arrange on top of cream cheese mixture and sprinkle with almonds',
 'bake at 350 for 45-55 minutes , or until tester comes out clean']

In [13]:
for step in dataset:
    print(re.sub(r' / ', '/', step))

mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


In [14]:
steps_ = '\n'.join(dataset)
pattern = re.compile(r'\d+\s{1}/\s{1}\d+')
res = re.findall(pattern, '\n'.join(dataset))
new_res = list(map(lambda x: x.strip().replace(" ", ""), res))
for index in range(len(res)):
    steps_ = steps_.replace(res[index], new_res[index])

print(steps_)

mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

In [15]:
with open('data/steps_sample.xml', 'r') as fp:
    recipes = BeautifulSoup(fp, 'xml').find_all('recipe')
recipes

[<recipe>
 <id>44123</id>
 <steps>
 <step has_minutes="1">in 1 / 4 cup butter , saute carrots , onion , celery and broccoli stems for 5 minutes</step>
 <step>add thyme , oregano and basil</step>
 <step has_minutes="1">saute 5 minutes more</step>
 <step>add wine and deglaze pan</step>
 <step>add hot chicken stock and reduce by one-third</step>
 <step>add worcestershire sauce , tabasco , smoked chicken , beans and broccoli florets</step>
 <step has_minutes="1">simmer 5 minutes</step>
 <step has_minutes="1">add cream , simmer 5 minutes more and season to taste</step>
 <step>drop in remaining butter , piece by piece , stirring until melted and serve immediately</step>
 <step>smoked chicken: on a covered grill , slightly smoke boneless chicken , cooking to medium rare</step>
 <step>chef meskan uses applewood chips and does not allow the grill to become too hot</step>
 </steps>
 </recipe>,
 <recipe>
 <id>67664</id>
 <steps>
 <step>mix all the ingredients using a blender</step>
 <step>pour in

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [16]:
set_word = set()
for elem in recipes:
    set_word.update(nltk.word_tokenize(elem.steps.text.strip()))
len(set_word)

19604

In [17]:
set_word

{'acidulated',
 '360',
 'scaloppini',
 'teaspoon',
 'array',
 'when',
 'lime-sugar',
 'frame',
 'blinis',
 'scar',
 'forcefully',
 'installments',
 'relaxes',
 'blended',
 'tongs',
 'eliminated',
 'yields',
 'eep',
 '269411',
 'seerve',
 '90',
 'thanksgiving',
 'crust-side',
 '300of',
 'boullion',
 'appraises',
 'parmesaon',
 'digestive',
 'fileter',
 'bullion',
 'sophisticated',
 'firmed',
 'un-greased',
 'noodles',
 'lighty',
 'older',
 'towls',
 'isk',
 'chiorzo',
 '7~10',
 'gingered',
 'on-and-off',
 'marlalade',
 'chenin',
 'cornstarch-water',
 'microplane',
 'boogie-dancing',
 'pangrattato',
 'reveals',
 'mad',
 'helpful',
 'scallions',
 'bibb',
 'elasticity',
 'irish',
 'pistashios',
 'rested',
 'mits',
 'pine-nuts',
 'zero',
 'varying',
 'macaromi',
 '400ml',
 'warp',
 'faxseed',
 'garlice',
 'unable',
 'sumac',
 'grandaughter',
 'speed',
 'davinici',
 '11inch',
 '500-degree',
 'gadget',
 'optionals',
 'banquet',
 'milky',
 'orange-juice',
 'bbw',
 'made1',
 'barrel',
 'pie-pan

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [18]:
df = dataframe['description']
dataframe['description'] = dataframe['description'].fillna("No desc")
dataframe['description'].isnull().sum()

0

In [19]:
dataset = dataframe.description.apply(nltk.sent_tokenize)
data = dataset.to_dict()
data

{0: ["an original recipe created by chef scott meskan, george's at the cove.",
  'we enjoyed this when we visited this restaurant in la jolla, california.',
  'this recipe is requested so often, they have it printed and ready at the hostess stand.',
  "it's unbeatable at the restaurant, but i do a pretty good job at home, too, if i do say so myself!"],
 1: ['my children and their friends ask for my homemade popsicles morning, noon and night.',
  'i never turn them down; who am i to tell them that they are good for them!',
  'for variety i substitute different flavours of frozen juice - grape, fruit punch, tropical etc.'],
 2: ['these were so go, it surprised even me.'],
 3: ['my sister-in-law made these for us at a family get together.',
  'they are delicious!!',
  'a little messy to make but worth the effort!!',
  'have a helper and make an'],
 4: ['i think a fondue is a very romantic casual dinner or wonderful for after the theatre snack served with a robust red wine.',
  'for dinner

In [20]:
top_5 = sorted(data, key = lambda x: len(data[x]), reverse=True)[:5]
top_5

[18408, 481, 22566, 6779, 16296]

In [21]:
for element in top_5:
    print(f"Результат длины: {len(data[element])}\nНаполнение элемента:\n{data[element]}\n" + 90*'–')

Результат длины: 76
Наполнение элемента:
['this wonderful icing is used for icing cakes and cookies as well as for borders and art work on cakes.', 'it makes a delicious filling also between the layers of cakes and under fondant icing.', 'you can make roses but it takes 3 or more days to dry them depending on the humidity.', 'there are many versions of “buttercream” icing.', 'some are made with eggs and all butter.', 'some varieties, you have to cook your sugar to a softball stage.', 'others are 100% shortening or a combination of shortening and butter.', 'each decorator has his or her favorite.', 'i personally think that the best taste and textured recipe is the one that has you cook your sugar, add to whipped eggs and use pounds of butter per batch.', 'but….', 'i live in a state that can easily be a 100 degrees for days on end during the summer and you know what butter does on hot days.', 'it melts!', 'a greasy puddle of melted icing on a cake plate is not something i want to look at

In [22]:
dataframe.iloc[top_5]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [23]:
test_value = nltk.pos_tag(nltk.word_tokenize(dataframe[dataframe['id'] == 241106].name.values[0]))
test_value

[('eggplant', 'JJ'),
 ('steaks', 'NNS'),
 ('with', 'IN'),
 ('chickpeas', 'NNS'),
 ('feta', 'VBP'),
 ('cheese', 'JJ'),
 ('and', 'CC'),
 ('black', 'JJ'),
 ('olives', 'NNS')]

In [25]:
test_new_value = [(value[-1], value[-2], max(len(value[0]), len(value[1]))) for value in test_value]
test_new_value

[('JJ', 'eggplant', 8),
 ('NNS', 'steaks', 6),
 ('IN', 'with', 4),
 ('NNS', 'chickpeas', 9),
 ('VBP', 'feta', 4),
 ('JJ', 'cheese', 6),
 ('CC', 'and', 3),
 ('JJ', 'black', 5),
 ('NNS', 'olives', 6)]

In [27]:
print(''.join([f"{ind[0]:^{ind[2] + 1}}" for ind in test_new_value]) + '\n' + ''.join([f"{ind[1]:^{ind[2] + 1}}" for ind in test_new_value]))

   JJ      NNS   IN     NNS     VBP   JJ    CC   JJ    NNS  
eggplant steaks with chickpeas feta cheese and black olives 
